In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns # Retirar depois
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

In [84]:
champions = pd.read_csv('data/championsdata.csv')
subchampions = pd.read_csv('data/runnerupsdata.csv')
total = pd.concat([champions, subchampions], ignore_index=True)
total.drop(["Win"], axis=1)
total

# 1. Preparar las variables (X) y el objetivo (y)
# ---------------------------------------------------------

# X = Todo MENOS 'Win'. 
# IMPORTANTE: También solemos quitar identificadores de texto como 'Team' 
# y a veces 'Year' o 'Game' para que el modelo se centre solo en estadísticas.
# Si dejas 'Team' (texto), los modelos matemáticos darán error.
X = total.drop(['Win', 'Team', 'Year', 'Game'], axis=1)

# y = SOLO la columna 'Win' (Ground Truth)
y = total['Win']

# 2. Dividir en Train y Test
# ---------------------------------------------------------
# test_size=0.2 significa que guardamos el 20% de los datos para el examen final
# random_state=42 sirve para que la división sea siempre igual cada vez que ejecutes (reproducibilidad)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)

# Comprobación rápida de tamaños
print(f"Entrenamiento: {X_train.shape} (Filas, Columnas)")
print(f"Test: {X_test.shape} (Filas, Columnas)")


Entrenamiento: (374, 20) (Filas, Columnas)
Test: (66, 20) (Filas, Columnas)


In [39]:
# LIMPIEZA VALORES NULOS

# TP - Tiros de campo de 3 puntos anotados
# TPA - Intentos de tres puntos
# TPP - porcentaje de tres puntos
#   TPP = TP / TPA
#   Si TPA == 0 entonces TPP es NaN

def get_percentage_nan_per_column(df):
  # retorna un df amb el % de NaNs de cada columna
  return df.isna().sum().sort_values()/len(df)*100.

percentage_nan = get_percentage_nan_per_column(total)
percentage_nan

total.fillna(0, inplace=True)

In [85]:
### CREAR ATRIBUT OVERTIME

# MP conte minuts jugats
# Valors trobats als df:

# MP     count      significat
# 240    339        duracio normal (no overtime)
# 265     33        1 overtime o prorroga
# 315      1        3 overtime o prorroga
# 40       1        valor incorrecte, si comprovem el mateix partit en els dos dfs els minuts reals son 240

# partir dels 240 minuts que dura un partit, cada 25 minuts més es considera que s'ha jugat un overtime


# Mostrar los valores únicos de la columna 'MP'
valores_unicos = total['MP'].unique()
print("Valores únicos en la columna MP:")
print(valores_unicos)

# Opcional: mostrar también cuántas veces aparece cada valor
print("\nConteo de cada valor:")
print(total['MP'].value_counts())

# Filtrar las filas
filas_filtradas = total[total['MP'] == 40]

# Mostrar las filas filtradas
print("Fila amb valor MP 40 erroni")
print(filas_filtradas)

# Filtrar la fila corresponent amb mateix Year i Game però diferent Team
fila_relacionada = total[
    (total['Year'] == filas_filtradas.iloc[0]['Year']) &
    (total['Game'] == filas_filtradas.iloc[0]['Game']) &
    (total['Team'] != filas_filtradas.iloc[0]['Team'])
]

# Mostrar el resultat
print("Fila del equip rival del valor MP 40 erroni")
print(fila_relacionada)
print("Temps real:", fila_relacionada["MP"].values[0])




Valores únicos en la columna MP:
[240 265 315  40]

Conteo de cada valor:
MP
240    401
265     36
315      2
40       1
Name: count, dtype: int64
Fila amb valor MP 40 erroni
     Year     Team  Game  Win  Home  MP  FG  FGA    FGP  TP  ...    FTP  ORB  \
263  1987  Celtics     3    1     1  40  42   86  0.488   1  ...  0.828   17   

     DRB  TRB  AST  STL  BLK  TOV  PF  PTS  
263   31   48   24    4    4   13  21  109  

[1 rows x 24 columns]
Fila del equip rival del valor MP 40 erroni
    Year    Team  Game  Win  Home   MP  FG  FGA    FGP  TP  ...    FTP  ORB  \
43  1987  Lakers     3    0     0  240  40   81  0.494   3  ...  0.741    8   

    DRB  TRB  AST  STL  BLK  TOV  PF  PTS  
43   24   32   18    8    4   10  24  103  

[1 rows x 24 columns]
Temps real: 240


In [86]:
####### SOLUCIO


X_train['OV'] = np.floor((X_train['MP'] - 240) / 25)
X_train['OV'] = X_train['OV'].apply(lambda x: max(x, 0)).astype(int)

X_train[['MP', 'OV']].head()
X_train.head()

#eliminar atribut MP
X_train = X_train.drop('MP', axis=1)
X_train.head()

fila_error_ov = X_train[
    (X_train['PTS'] == filas_filtradas.iloc[0]['PTS']) &
    (X_train['AST'] == filas_filtradas.iloc[0]['AST']) &
    (X_train['BLK'] == filas_filtradas.iloc[0]['BLK'])
]

# Mostrar el valor de OV
if fila_error_ov.empty:
    print("No existeix aquesta fila a X_train")
else:
    print("OV de la fila que tenia MP = 40:", fila_error_ov)

,Home,FG,FGA,FGP,TP,TPA,TPP,FT,FTA,FTP,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,OV
263,1,42,86,0.488,1,5,0.2,24,29,0.828,17,31,48,24,4,4,13,21,109,0


MP   - Minutos jugados -> Cambiar por categoria OVERTIME (1 SI - 0 NO)
Game - Numero de partido de la serie -> INNECESARIO
Year - Año de la serie -> INNECESARIO
Team - Nombre del equipo -> ¿INNECESARIO? Analizar

# 1. La variable a predecir (Target)
Sin duda alguna, la variable objetivo es:

Win (0 o 1).

Es un problema clásico de clasificación binaria: queremos saber qué factores aumentan la probabilidad de que esa variable sea 1.

# 2. Las variables que más influyen (Hipótesis)
Si tuviera que apostar cuáles tendrán una correlación más fuerte con la victoria (excluyendo PTS, que es el resultado directo), diría que son estas cuatro, en este orden:

## 1. FGP (Porcentaje de Tiros de Campo) y TPP (Triples)
Es la métrica de eficiencia más pura. En la NBA, el equipo que tira con mejor porcentaje suele ganar.

Lógica: No importa cuántos tiros intentes (FGA), si no entran, no sumas. Un FGP alto suele ser el indicador #1 de victoria.

## 2. TOV (Pérdidas de Balón - Turnovers)
Esta debería tener una correlación negativa.

Lógica: Cada pérdida es una oportunidad de tiro que tu equipo pierde y (generalmente) una oportunidad fácil de contraataque para el rival. Quien cuida mejor el balón, suele ganar las finales.

## 3. Home (Factor Cancha)
En la NBA, y especialmente en las finales, jugar en casa es una ventaja estadística masiva.

Lógica: El público, el descanso, no viajar y el arbitraje (a veces) influyen. Espero ver una correlación positiva fuerte aquí.

## 4. DRB (Rebotes Defensivos)
Más que los ofensivos, los defensivos son clave.

Lógica: Si aseguras el rebote defensivo, evitas que el rival tenga "segundas oportunidades". "La defensa gana campeonatos", y el rebote es parte de la defensa.

⚠️ La "trampa" de los Puntos (PTS)
Técnicamente, PTS es la variable que más influye (si tienes más puntos que el rival, ganas el 100% de las veces). Pero en un modelo predictivo, a veces se excluye porque es una redundancia de la victoria.